# 하이퍼 매개 변수 튜닝

다수의 기계 학습 알고리즘에는 *하이퍼 매개 변수*가 필요합니다. 하이퍼 매개 변수란 학습에는 영향을 주지만 학습 데이터 자체에서는 확인할 수 없는 매개 변수 값입니다. 예를 들어 로지스틱 회귀 모델 학습 시에는 *정규화 비율* 하이퍼 매개 변수를 사용하여 모델의 바이어스를 완화할 수 있습니다. 그리고 콘볼루션 신경망 학습 시에는 *학습 속도*, *일괄 처리 크기* 등의 하이퍼 매개 변수를 사용하여 가중치 조정 방식과 미니 배치에서 처리되는 데이터 항목의 수를 각각 제어할 수 있습니다. 선택하는 하이퍼 매개 변수 값에 따라 학습된 모델의 성능이나 모델을 학습시키는 데 걸리는 시간이 크게 달라질 수 있으며, 여러 매개 변수 조합을 적용하여 최적의 값을 찾아야 하는 경우가 많습니다.

여기서는 하이퍼 매개 변수가 두 개인 분류 모델을 학습시킵니다. 하지만 이 모델의 원칙은 Azure Machine Learning을 사용하여 학습시킬 수 있는 모든 종류의 모델에 적용할 수 있습니다.

## 작업 영역에 연결

이 Notebook의 작업을 시작하려면 먼저 작업 영역에 연결합니다.

> **참고**: Azure 구독에 인증된 세션을 아직 설정하지 않은 경우에는 링크를 클릭하고 인증 코드를 입력한 다음 Azure에 로그인하여 인증하라는 메시지가 표시됩니다.

In [ ]:
import azureml.core
from azureml.core import Workspace

# 저장된 구성 파일에서 작업 영역 로드
ws = Workspace.from_config()
print('Ready to use Azure ML {} to work with {}'.format(azureml.core.VERSION, ws.name))

## 데이터 준비

이 랩에서는 당뇨병 환자의 세부 정보가 포함된 데이터 세트를 사용합니다. 아래 셀의 명령을 실행하여 이 데이터 세트를 만듭니다. 데이터 세트가 이미 있으면 기존 버전이 사용됩니다.

In [ ]:
from azureml.core import Dataset

default_ds = ws.get_default_datastore()

if 'diabetes dataset' not in ws.datasets:
    default_ds.upload_files(files=['./data/diabetes.csv', './data/diabetes2.csv'], # Upload the diabetes csv files in /data
                        target_path='diabetes-data/', # Put it in a folder path in the datastore
                        overwrite=True, # Replace existing files of the same name
                        show_progress=True)

    #Create a tabular dataset from the path on the datastore (this may take a short while)
    tab_data_set = Dataset.Tabular.from_delimited_files(path=(default_ds, 'diabetes-data/*.csv'))

    # Register the tabular dataset
    try:
        tab_data_set = tab_data_set.register(workspace=ws, 
                                name='diabetes dataset',
                                description='diabetes data',
                                tags = {'format':'CSV'},
                                create_new_version=True)
        print('Dataset registered.')
    except Exception as ex:
        print(ex)
else:
    print('Dataset already registered.')

## 학습 스크립트 준비

이제 모델을 학습시키는 데 사용할 학습 스크립트용 폴더를 만들어 보겠습니다.

In [ ]:
import os

experiment_folder = 'diabetes_training-hyperdrive'
os.makedirs(experiment_folder, exist_ok=True)

print('Folder ready.')

이제 모델 학습을 위한 Python 스크립트를 만듭니다. 이 예제에서는 *경사 부스팅* 알고리즘을 사용하여 분류 모델을 학습시킵니다. 이 스크립트에는 다음 항목이 포함되어야 합니다.

- 최적화할 각 하이퍼 매개 변수의 인수(여기서는 경사 부스팅 알고리즘용 예측 도구의 수와 학습 속도)
- 최적화할 성능 메트릭을 기록하는 코드. 여기서는 AUC와 정확도를 모두 기록하므로, AUC와 정확도 중 하나를 최대한 높이도록 모델을 최적화할 수 있습니다.

In [ ]:
%%writefile $experiment_folder/diabetes_training.py
# 라이브러리 가져오기
import argparse, joblib, os
from azureml.core import Run
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.metrics import roc_auc_score, roc_curve

# 실험 실행 컨텍스트 가져오기
run = Run.get_context()

# 스크립트 인수 가져오기
parser = argparse.ArgumentParser()

# 입력 데이터 세트
parser.add_argument("--input-data", type=str, dest='input_data', help='training dataset')

# 하이퍼 매개 변수
parser.add_argument('--learning_rate', type=float, dest='learning_rate', default=0.1, help='learning rate')
parser.add_argument('--n_estimators', type=int, dest='n_estimators', default=100, help='number of estimators')

# 인수 모음에 인수 추가
args = parser.parse_args()

# 하이퍼 매개 변수 값 로깅
run.log('learning_rate',  np.float(args.learning_rate))
run.log('n_estimators',  np.int(args.n_estimators))

# 당뇨병 데이터 세트 로드
print("Loading Data...")
diabetes = run.input_datasets['training_data'].to_pandas_dataframe() # Get the training data from the estimator input

# 기능 및 레이블 분리
X, y = diabetes[['Pregnancies','PlasmaGlucose','DiastolicBloodPressure','TricepsThickness','SerumInsulin','BMI','DiabetesPedigree','Age']].values, diabetes['Diabetic'].values

# 데이터를 학습 세트와 테스트 세트로 분할
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.30, random_state=0)

# 지정된 하이퍼 매개 변수로 Gradient Boosting 분류 모델 학습
print('Training a classification model')
model = GradientBoostingClassifier(learning_rate=args.learning_rate,
                                   n_estimators=args.n_estimators).fit(X_train, y_train)

# 정확도 계산
y_hat = model.predict(X_test)
acc = np.average(y_hat == y_test)
print('Accuracy:', acc)
run.log('Accuracy', np.float(acc))

# AUC 계산
y_scores = model.predict_proba(X_test)
auc = roc_auc_score(y_test,y_scores[:,1])
print('AUC: ' + str(auc))
run.log('AUC', np.float(auc))

# 실행 출력에 모델 저장
os.makedirs('outputs', exist_ok=True)
joblib.dump(value=model, filename='outputs/diabetes_model.pkl')

run.complete()

## 컴퓨팅 컨텍스트 만들기

하이퍼 매개 변수를 튜닝할 때는 하이퍼 매개 변수 값이 서로 다른 학습 반복을 여러 번 실행한 다음 결과 모델의 성능 메트릭을 비교합니다. 여기서는 이 과정을 효율적으로 진행하기 위해 주문형 클라우드 컴퓨팅을 활용하여 클러스터를 만듭니다. 그러면 여러 학습 반복을 동시에 실행할 수 있습니다.

다음 코드를 사용하여 Azure Machine Learning 컴퓨팅 클러스터를 지정합니다. 이 클러스터는 아직 없으면 자동으로 생성됩니다.

> **중요**: 컴퓨팅 클러스터를 실행하기 전에 아래 코드에서 *your-compute-cluster*를 컴퓨팅 클러스터의 이름으로 변경하세요! 클러스터 이름은 2~16자 사이의 전역으로 고유한 이름이어야 합니다. 유효한 문자는 영문자, 숫자 및 문자입니다.

In [ ]:
from azureml.core.compute import ComputeTarget, AmlCompute
from azureml.core.compute_target import ComputeTargetException

cluster_name = "your-compute-cluster"

try:
    # Check for existing compute target
    training_cluster = ComputeTarget(workspace=ws, name=cluster_name)
    print('Found existing cluster, use it.')
except ComputeTargetException:
    # If it doesn't already exist, create it
    try:
        compute_config = AmlCompute.provisioning_configuration(vm_size='STANDARD_DS11_V2', max_nodes=2)
        training_cluster = ComputeTarget.create(ws, cluster_name, compute_config)
        training_cluster.wait_for_completion(show_output=True)
    except Exception as ex:
        print(ex)
    

> **참고**: 컴퓨팅 인스턴스와 클러스터는 표준 Azure 가상 머신 이미지를 기반으로 합니다. 이 연습에서는 비용과 성능 간 최적의 균형을 달성하기 위해 *Standard_DS11_v2* 이미지를 사용하는 것이 좋습니다. 구독의 할당량이 적어 이 이미지를 포함할 수 없는 경우 대체 이미지를 선택할 수 있습니다. 그러나 큰 이미지는 높은 비용을 야기할 수 있고 작은 이미지는 작업을 완료하는 데 충분하지 않을 수 있으므로 신중히 선택하는 것이 좋습니다. Azure 관리자에게 요청하여 할당량을 늘릴 수도 있습니다.

컴퓨터에서 호스트할 Python 환경이 필요하므로 이 Python 환경을 Conda 사양 파일로 정의하겠습니다.

In [ ]:
%%writefile $experiment_folder/hyperdrive_env.yml
name: batch_environment
dependencies:
- python=3.6.2
- scikit-learn
- pip
- pip:
  - azureml-defaults


## 하이퍼 매개 변수 튜닝 실험 실행

Azure Machine Learning에는 *Hyperdrive* 실험을 통한 하이퍼 매개 변수 튜닝 기능이 포함되어 있습니다. 이러한 실험에서는 하이퍼 매개 변수 조합이 각기 다른 여러 하위 실행이 시작됩니다. 따라서 성능이 가장 우수한 모델을 생성하는 실행을 확인한 다음 이 실행에서 학습된 모델을 등록 및 배포용으로 선택할 수 있습니다. 여기서 성능이 가장 우수한 모델은 기록된 대상 성능 메트릭 중 최적화하려는 메트릭을 기준으로 결정됩니다.

> **참고**: 이 예제에서는 조기 중지 정책을 지정하지 않습니다. 학습 스크립트가 여러 학습 반복을 수행하여 각 반복의 기본 메트릭을 로깅하는 경우에만 이 정책을 지정하면 됩니다. 여러 *Epoch*에 걸쳐 심층 신경망 모델을 학습시킬 때 대개 이 방식을 사용합니다.

In [ ]:
from azureml.core import Experiment, ScriptRunConfig, Environment
from azureml.train.hyperdrive import GridParameterSampling, HyperDriveConfig, PrimaryMetricGoal, choice
from azureml.widgets import RunDetails

# 실험용 Python 환경 만들기
hyper_env = Environment.from_conda_specification("experiment_env", experiment_folder + "/hyperdrive_env.yml")

# 학습 데이터 세트 가져오기
diabetes_ds = ws.datasets.get("diabetes dataset")

# 스크립트 구성 만들기
script_config = ScriptRunConfig(source_directory=experiment_folder,
                                script='diabetes_training.py',
                                # Add non-hyperparameter arguments -in this case, the training dataset
                                arguments = ['--input-data', diabetes_ds.as_named_input('training_data')],
                                environment=hyper_env,
                                compute_target = training_cluster)

# 매개 변수 값 범위 샘플링
params = GridParameterSampling(
    {
        # Hyperdrive will try 6 combinations, adding these as script arguments
        '--learning_rate': choice(0.01, 0.1, 1.0),
        '--n_estimators' : choice(10, 100)
    }
)

# 하이퍼 드라이브 설정 구성
hyperdrive = HyperDriveConfig(run_config=script_config, 
                          hyperparameter_sampling=params, 
                          policy=None, # No early stopping policy
                          primary_metric_name='AUC', # Find the highest AUC metric
                          primary_metric_goal=PrimaryMetricGoal.MAXIMIZE, 
                          max_total_runs=6, # Restict the experiment to 6 iterations
                          max_concurrent_runs=2) # Run up to 2 iterations in parallel

# 실험 실행
experiment = Experiment(workspace=ws, name='mslearn-diabetes-hyperdrive')
run = experiment.submit(config=hyperdrive)

# 실험이 실행될 때 Notebook의 상태 표시
RunDetails(run).show()
run.wait_for_completion()

위의 위젯에서 실험 실행 상태를 확인할 수 있습니다. [Azure Machine Learning Studio](https://ml.azure.com)에서 기본 하이퍼 드라이브 실험 실행 및 해당 하위 실험을 확인할 수도 있습니다.

> **참고**: 숫자가 아닌 값은 시각화할 수 없다는 메시지가 표시되는 경우 무시하면 됩니다.

## 성능이 가장 우수한 실행 결정

모든 실행이 완료되면 지정한 성능 메트릭을 기준으로 하여 성능이 가장 우수한 실행(여기서는 AUC가 가장 높은 실행)을 찾을 수 있습니다.

In [ ]:
# 기본 메트릭으로 정렬된 모든 하위 실행 인쇄
for child_run in run.get_children_sorted_by_primary_metric():
    print(child_run)

# 최고의 실행과 해당 메트릭 및 인수 가져오기
best_run = run.get_best_run_by_primary_metric()
best_run_metrics = best_run.get_metrics()
script_arguments = best_run.get_details() ['runDefinition']['arguments']
print('Best Run Id: ', best_run.id)
print(' -AUC:', best_run_metrics['AUC'])
print(' -Accuracy:', best_run_metrics['Accuracy'])
print(' -Arguments:',script_arguments)

성능이 가장 우수한 실행을 찾은 후에는 이 실행에서 학습된 모델을 등록할 수 있습니다.

In [ ]:
from azureml.core import Model

# 모델 등록
best_run.register_model(model_path='outputs/diabetes_model.pkl', model_name='diabetes_model',
                        tags={'Training context':'Hyperdrive'},
                        properties={'AUC': best_run_metrics['AUC'], 'Accuracy': best_run_metrics['Accuracy']})

# 등록된 모델 목록 표시
for model in Model.list(ws):
    print(model.name, 'version:', model.version)
    for tag_name in model.tags:
        tag = model.tags[tag_name]
        print ('\t',tag_name, ':', tag)
    for prop_name in model.properties:
        prop = model.properties[prop_name]
        print ('\t',prop_name, ':', prop)
    print('\n')

> **추가 정보**: 하이퍼 드라이브에 대한 자세한 내용은 [Azure ML 설명서](https://docs.microsoft.com/azure/machine-learning/how-to-tune-hyperparameters)를 참조하세요.